In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score

In [3]:
train = pd.read_csv('train.csv', encoding = "ISO-8859-1")
test = pd.read_csv('test.csv')
addresses = pd.read_csv('addresses.csv')
latlons = pd.read_csv('latlons.csv')

C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
test.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200,20,10,20.0,0,0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000,20,10,100.0,0,0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100,20,10,10.0,0,0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200,20,10,20.0,0,0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100,20,10,10.0,0,0,140.0,NaN


In [5]:
len(train)

250306

In [6]:
train = train[np.isfinite(train['compliance'])]

In [7]:
len(train)

159880

In [8]:
train.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance'],
      dtype='object')

In [9]:
train = train[train['country'] == 'USA']
test = test[test['country'] == 'USA']

In [10]:
len(train)

159869

In [11]:
addresses.head()

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [12]:
latlons.head()

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [13]:
train = pd.merge(train, pd.merge(addresses, latlons, on = 'address'), on = 'ticket_id')

In [14]:
test = pd.merge(test, pd.merge(addresses, latlons, on = 'address'), on = 'ticket_id')

In [15]:
train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,address,lat,lon
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"2900 tyler, Detroit MI",42.390729,-83.124268
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,75.0,6/2/2005 0:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0,"4311 central, Detroit MI",42.326937,-83.135118
2,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"6478 northfield, Detroit MI",42.145257,-83.208233
3,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"8027 brentwood, Detroit MI",42.433466,-83.023493
4,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"8228 mt elliott, Detroit MI",42.388641,-83.037858


In [16]:
train

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,address,lat,lon
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"2900 tyler, Detroit MI",42.390729,-83.124268
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,75.0,6/2/2005 0:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0,"4311 central, Detroit MI",42.326937,-83.135118
2,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"6478 northfield, Detroit MI",42.145257,-83.208233
3,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"8027 brentwood, Detroit MI",42.433466,-83.023493
4,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"8228 mt elliott, Detroit MI",42.388641,-83.037858
5,18733,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"8228 mt elliott, Detroit MI",42.388641,-83.037858
6,28204,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Inc, Nanno",15307,SEVEN MILE,NaN,1537.0,E. Seven Mile,Detroit,...,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"15307 seven mile, Detroit MI",42.435773,-82.963348
7,18743,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Gardner Resale, GAR",9100,VAN DYKE,NaN,91.0,Van Dyke,Detroit,...,855.0,NaN,NO PAYMENT APPLIED,IN COLLECTION,NaN,non-compliant by no payment,0.0,"9100 van dyke, Detroit MI",42.395765,-83.022333
8,18741,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Hardaway, Kevin",20024,SCHAEFER,NaN,224.0,Schaefer,Detroit,...,855.0,NaN,NO PAYMENT APPLIED,IN COLLECTION,NaN,non-compliant by no payment,0.0,"20024 schaefer, Detroit MI",42.440190,-83.180488
9,18978,"Buildings, Safety Engineering & Env Department","Williams, Darrin","TLC Hand Car Wash, a/k/a",9425,VAN DYKE,NaN,9425.0,Van Dyke,Detroit,...,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"9425 van dyke, Detroit MI",42.399222,-83.023161


In [17]:
train.drop(['agency_name', 'inspector_name', 'violator_name', 'non_us_str_code', 'violation_description', 'grafitti_status', 'state_fee', 'admin_fee', 'ticket_issued_date', 'hearing_date','payment_amount', 'balance_due', 'payment_date', 'payment_status','collection_status', 'compliance_detail', 'violation_zip_code', 'country', 'address', 'violation_street_number','violation_street_name', 'mailing_address_str_number', 'mailing_address_str_name', 'city', 'state', 'zip_code', 'address'], axis=1, inplace=True)

In [18]:
train.head()

,ticket_id,violation_code,disposition,fine_amount,late_fee,discount_amount,clean_up_cost,judgment_amount,compliance,lat,lon
0,22056,9-1-36(a),Responsible by Default,250.0,25.0,0,0,305.0,0.0,42.390729,-83.124268
1,27586,61-63.0600,Responsible by Determination,750.0,75.0,0,0,855.0,1.0,42.326937,-83.135118
2,22046,9-1-36(a),Responsible by Default,250.0,25.0,0,0,305.0,0.0,42.145257,-83.208233
3,18738,61-63.0500,Responsible by Default,750.0,75.0,0,0,855.0,0.0,42.433466,-83.023493
4,18735,61-63.0100,Responsible by Default,100.0,10.0,0,0,140.0,0.0,42.388641,-83.037858


In [19]:
label_encoder = LabelEncoder()

In [20]:
label_encoder.fit(train['disposition'].append(test['disposition'], ignore_index = True))

LabelEncoder()

In [21]:
label_encoder.classes_

array(['Responsible (Fine Waived) by Admis',
       'Responsible (Fine Waived) by Deter',
       'Responsible - Compl/Adj by Default',
       'Responsible - Compl/Adj by Determi', 'Responsible by Admission',
       'Responsible by Default', 'Responsible by Determination',
       'Responsible by Dismissal'], dtype=object)

In [22]:
train['disposition'] = label_encoder.transform(train['disposition'])

In [23]:
train['disposition']

0         5
1         6
2         5
3         5
4         5
5         5
6         5
7         5
8         5
9         5
10        6
11        6
12        5
13        5
14        5
15        4
16        5
17        5
18        5
19        5
20        5
21        5
22        6
23        5
24        5
25        5
26        4
27        4
28        5
29        5
         ..
159839    5
159840    5
159841    4
159842    5
159843    5
159844    5
159845    5
159846    5
159847    5
159848    5
159849    5
159850    5
159851    1
159852    5
159853    5
159854    5
159855    5
159856    5
159857    5
159858    5
159859    5
159860    5
159861    5
159862    5
159863    5
159864    5
159865    5
159866    5
159867    6
159868    5
Name: disposition, Length: 159869, dtype: int32

In [24]:
test['disposition'] = label_encoder.transform(test['disposition'])

In [25]:
test['disposition']

0        5
1        5
2        5
3        5
4        5
5        5
6        5
7        6
8        5
9        5
10       5
11       5
12       5
13       5
14       5
15       5
16       5
17       5
18       5
19       5
20       5
21       5
22       5
23       5
24       5
25       5
26       5
27       5
28       5
29       5
        ..
60971    5
60972    5
60973    6
60974    6
60975    5
60976    5
60977    5
60978    5
60979    4
60980    5
60981    5
60982    4
60983    4
60984    6
60985    6
60986    5
60987    5
60988    5
60989    5
60990    5
60991    5
60992    5
60993    5
60994    5
60995    5
60996    5
60997    5
60998    5
60999    5
61000    5
Name: disposition, Length: 61001, dtype: int32

In [26]:
label_encoder = LabelEncoder()

In [28]:
label_encoder.fit(train['violation_code'].append(test['violation_code'], ignore_index = True))

LabelEncoder()

In [29]:
train['violation_code'] = label_encoder.transform(train['violation_code'])

In [30]:
test['violation_code'] = label_encoder.transform(test['violation_code'])

In [31]:
test['violation_code']

0         38
1         57
2         25
3         57
4         27
5         57
6        128
7         57
8         57
9         57
10       122
11        57
12        57
13       122
14        27
15        57
16        57
17        57
18        57
19       175
20        92
21        57
22        57
23       122
24        57
25        57
26        57
27        57
28        25
29        57
        ... 
60971     92
60972    175
60973    175
60974     95
60975     92
60976    175
60977    175
60978    175
60979    175
60980    175
60981    175
60982    175
60983    175
60984     86
60985    175
60986    128
60987    130
60988     55
60989    175
60990    182
60991    221
60992    182
60993    182
60994    182
60995    213
60996    184
60997    184
60998     27
60999     27
61000    122
Name: violation_code, Length: 61001, dtype: int32

In [32]:
label_encoder.classes_

array(['19410901', '19420901', '19450901', '19830901', '19840901',
       '19850901', '19910901', '20130901', '20160901', '20180901',
       '22-2-16', '22-2-17', '22-2-17(a)', '22-2-18', '22-2-20',
       '22-2-21(a)', '22-2-21(b)', '22-2-22', '22-2-22(a)', '22-2-23',
       '22-2-38', '22-2-41', '22-2-41(b)', '22-2-42', '22-2-42(b)',
       '22-2-43', '22-2-44', '22-2-45', '22-2-48(a)', '22-2-48(b)',
       '22-2-49', '22-2-49(a)', '22-2-49(b)', '22-2-49(c)', '22-2-49(d)',
       '22-2-53', '22-2-55', '22-2-56', '22-2-61', '22-2-83',
       '22-2-83(a)(b)(c)', '22-2-83(a)(c)', '22-2-83(b)', '22-2-83(c)',
       '22-2-83(d)', '22-2-83(d)(e)', '22-2-83(e)',
       '22-2-84 (DO NOT USE ', '22-2-84(a)', '22-2-84(a)(1)(2)(4)',
       '22-2-84(b)(1)', '22-2-84(b)(3)', '22-2-85', '22-2-87',
       '22-2-87(a)', '22-2-88', '22-2-88(a)', '22-2-88(b)', '22-2-88(c)',
       '22-2-91', '22-2-92', '22-2-93', '22-2-94', '22-2-96', '22-2-97',
       '22-2-97(b)', '22-3-2', '61-100.0100', '61-101.01

In [33]:
train['lat'] = train['lat'].fillna(train['lat'].mean())

In [35]:
train['lon'] = train['lon'].fillna(train['lon'].mean())

In [36]:
test['lat'] = test['lat'].fillna(test['lat'].mean())

In [37]:
test['lon'] = test['lon'].fillna(test['lon'].mean())

In [40]:
train_columns = list(train.columns.values)

In [41]:
train_columns.remove('compliance')

In [42]:
test = test[train_columns]

In [43]:
X = train.ix[:, train.columns != 'compliance']

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [46]:
y = train['compliance']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [48]:
regr_rf = RandomForestRegressor()

In [49]:
grid_values = {'n_estimators': [10, 80], 'max_depth': [None, 20]}

In [53]:
grid_clf_auc = GridSearchCV(regr_rf, param_grid = grid_values, scoring = 'roc_auc')
grid_clf_auc.fit(X_train, y_train)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [10, 80], 'max_depth': [None, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [51]:
print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)

Grid best parameter (max. AUC):  {'max_depth': 20, 'n_estimators': 80}


In [52]:
print('Grid best score (AUC): ', grid_clf_auc.best_score_)

Grid best score (AUC):  0.8196696804211402


In [54]:
pd.DataFrame(grid_clf_auc.predict(test), test.ticket_id)

,0
ticket_id,
284932,0.000577
285362,0.003361
285361,0.413415
285338,0.324703
285346,0.738110
285345,0.324703
285347,0.273422
285342,0.912500
285530,0.000000
